In [24]:
import os
import pandas as pd

# Load the data into a DataFrame
file_path = "Case Study Data - Read Only - case_study_data_2025-01-16T06_49_12.19881Z.csv"
df = pd.read_csv(file_path)

In [25]:

print(df.info())

print(df.shape)


print(df.head())


print(df.describe())

print(df.columns.tolist())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333405 entries, 0 to 333404
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   DATE                 333405 non-null  object
 1   ANONYMIZED CATEGORY  333405 non-null  object
 2   ANONYMIZED PRODUCT   333405 non-null  object
 3   ANONYMIZED BUSINESS  333405 non-null  object
 4   ANONYMIZED LOCATION  333405 non-null  object
 5   QUANTITY             333405 non-null  int64 
 6   VALUE                333397 non-null  object
dtypes: int64(1), object(6)
memory usage: 17.8+ MB
None
(333405, 7)
                       DATE ANONYMIZED CATEGORY ANONYMIZED PRODUCT  \
0  August 18, 2024, 9:32 PM        Category-106       Product-21f4   
1  August 18, 2024, 9:32 PM        Category-120       Product-4156   
2  August 18, 2024, 9:32 PM        Category-121       Product-49bd   
3  August 18, 2024, 9:32 PM         Category-76       Product-61dd   
4  August 18, 2024, 9:

In [26]:
import pandas as pd
import numpy as np

def analyze_data_types(df):
    print("\n=== Column Data Types Analysis ===")
    
    # Identify numeric and categorical columns
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    
    print("\nNumerical Columns:")
    for col in numeric_cols:
        print(f"- {col}")
        
    print("\nCategorical Columns:")
    for col in categorical_cols:
        print(f"- {col}")
        
    return numeric_cols, categorical_cols

def check_data_quality(df):
    print("\n=== Data Quality Report ===")
    
    # Get total rows and columns
    print(f"\nDataset Shape: {df.shape} (rows, columns)")
    
    # 1. Missing Values
    print("\n1. Missing Values:")
    missing = df.isnull().sum()
    missing_percent = (missing / len(df)) * 100
    missing_info = pd.DataFrame({
        'Missing Count': missing,
        'Missing Percentage': missing_percent.round(2)
    })
    print(missing_info[missing_info['Missing Count'] > 0])
    
    # 2. Duplicates
    print("\n2. Duplicate Rows:")
    duplicates = df.duplicated().sum()
    print(f"Total duplicates: {duplicates} ({(duplicates/len(df))*100:.2f}%)")
    
    # 3. Basic Statistics for Numerical Columns
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    if len(numeric_cols) > 0:
        print("\n3. Numerical Columns Summary:")
        print(df[numeric_cols].describe())
    
    # 4. Unique Values in Categorical Columns
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    if len(categorical_cols) > 0:
        print("\n4. Categorical Columns Summary:")
        for col in categorical_cols:
            unique_count = df[col].nunique()
            print(f"\n{col}:")
            print(f"- Unique values: {unique_count}")
            if unique_count < 10:  # Only show value counts if fewer than 10 unique values
                print("- Value counts:")
                print(df[col].value_counts())
    # 5. Outliers (for numerical columns)
    print("3. Outliers (using IQR method):")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
        print(f"{col}: {len(outliers)} outliers detected")
    print("\n")

     # 6. Class Imbalance (if target variable is specified)
    print("6. Class Distribution (for categorical columns):")
    for col in categorical_cols:
        balance = df[col].value_counts(normalize=True)
        if len(balance) < 10:  # Only show if fewer than 10 unique values
            print(f"\n{col}:")
            print(balance.multiply(100).round(2).astype(str) + '%')
            print("\n")
    
check_data_quality(df)
# Example usage:
# df = pd.read_csv('your_data.csv')
# numeric_cols, categorical_cols = analyze_data_types(df)
 


=== Data Quality Report ===

Dataset Shape: (333405, 7) (rows, columns)

1. Missing Values:
       Missing Count  Missing Percentage
VALUE              8                 0.0

2. Duplicate Rows:
Total duplicates: 3524 (1.06%)

3. Numerical Columns Summary:
            QUANTITY
count  333405.000000
mean        2.321186
std         3.790614
min         0.000000
25%         1.000000
50%         1.000000
75%         2.000000
max       359.000000

4. Categorical Columns Summary:

DATE:
- Unique values: 96703

ANONYMIZED CATEGORY:
- Unique values: 46

ANONYMIZED PRODUCT:
- Unique values: 820

ANONYMIZED BUSINESS:
- Unique values: 4800

ANONYMIZED LOCATION:
- Unique values: 53

VALUE:
- Unique values: 1050
3. Outliers (using IQR method):
QUANTITY: 48631 outliers detected


6. Class Distribution (for categorical columns):


In [27]:
def check_missing_values(df):
    print("\n=== Missing Values Analysis ===")
    
    # Make a copy to avoid modifying original data
    df_temp = df.copy()
    
    # Convert VALUE to numeric, handling the commas
    df_temp['VALUE'] = df_temp['VALUE'].replace({',': ''}, regex=True)
    
    print("\nBefore cleaning:")
    print("Missing values in each column:")
    print(df_temp.isnull().sum())
    
    return df_temp

# Run it
df_temp = check_missing_values(df)


=== Missing Values Analysis ===

Before cleaning:
Missing values in each column:
DATE                   0
ANONYMIZED CATEGORY    0
ANONYMIZED PRODUCT     0
ANONYMIZED BUSINESS    0
ANONYMIZED LOCATION    0
QUANTITY               0
VALUE                  8
dtype: int64


In [28]:
def check_duplicates(df):
    print("\n=== Duplicates Analysis ===")
    
    # Count total duplicates
    dup_count = df.duplicated().sum()
    print(f"\nTotal number of duplicate rows: {dup_count}")
    
    # Show example of duplicates
    if dup_count > 0:
        print("\nFirst few duplicate rows:")
        duplicates = df[df.duplicated(keep='first')]
        print(duplicates.head())
        
        # Check which columns are most commonly duplicated
        print("\nDuplicate counts by column combinations:")
        for col in df.columns:
            dup_by_col = df.duplicated(subset=[col]).sum()
            print(f"{col}: {dup_by_col} duplicates")
    
    return duplicates

# Run it
duplicates = check_duplicates(df)


=== Duplicates Analysis ===

Total number of duplicate rows: 3524

First few duplicate rows:
                            DATE ANONYMIZED CATEGORY ANONYMIZED PRODUCT  \
6153   January 6, 2024, 11:52 AM         Category-91       Product-1b48   
7554       July 9, 2024, 2:26 PM        Category-104       Product-af50   
7555       July 9, 2024, 2:26 PM         Category-92       Product-d09a   
12238    April 19, 2024, 3:19 PM         Category-75       Product-086d   
12239    April 19, 2024, 3:19 PM        Category-106       Product-21f4   

      ANONYMIZED BUSINESS ANONYMIZED LOCATION  QUANTITY  VALUE  
6153        Business-20fc       Location-b125         1  3,680  
7554        Business-476c       Location-b27b         1  1,310  
7555        Business-476c       Location-b27b         1  1,550  
12238       Business-b48e       Location-03fc         3  2,090  
12239       Business-b48e       Location-03fc         2    850  

Duplicate counts by column combinations:
DATE: 236702 duplicates

In [29]:
def check_outliers(df):
    print("\n=== QUANTITY Outliers Analysis ===")
    
    # Basic statistics
    print("\nQUANTITY statistics:")
    print(df['QUANTITY'].describe())
    
    # Calculate outlier boundaries
    Q1 = df['QUANTITY'].quantile(0.25)
    Q3 = df['QUANTITY'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    print(f"\nOutlier boundaries:")
    print(f"Lower bound: {lower_bound}")
    print(f"Upper bound: {upper_bound}")
    
    # Find outliers
    outliers = df[(df['QUANTITY'] < lower_bound) | (df['QUANTITY'] > upper_bound)]
    print(f"\nNumber of outliers: {len(outliers)}")
    
    # Show distribution of outlier quantities
    print("\nOutlier QUANTITY values:")
    print(outliers['QUANTITY'].value_counts().head())
    
    return outliers

# Run it
outliers = check_outliers(df)


=== QUANTITY Outliers Analysis ===

QUANTITY statistics:
count    333405.000000
mean          2.321186
std           3.790614
min           0.000000
25%           1.000000
50%           1.000000
75%           2.000000
max         359.000000
Name: QUANTITY, dtype: float64

Outlier boundaries:
Lower bound: -0.5
Upper bound: 3.5

Number of outliers: 48631

Outlier QUANTITY values:
QUANTITY
5     18511
4      9834
10     8232
6      3262
7      1648
Name: count, dtype: int64


In [30]:
def clean_data_step_by_step(df):
    print("\n=== Cleaning Data Step by Step ===")
    df_clean = df.copy()
    
    # 1. Clean VALUE column
    print("\nStep 1: Cleaning VALUE column")
    df_clean['VALUE'] = df_clean['VALUE'].replace({',': ''}, regex=True)
    print("VALUE column cleaned")
    
    # 2. Remove duplicates
    print("\nStep 2: Removing duplicates")
    initial_rows = len(df_clean)
    df_clean = df_clean.drop_duplicates()
    rows_removed = initial_rows - len(df_clean)
    print(f"Removed {rows_removed} duplicate rows")
    
    # 3. Flag outliers
    print("\nStep 3: Flagging outliers")
    Q1 = df_clean['QUANTITY'].quantile(0.25)
    Q3 = df_clean['QUANTITY'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    df_clean['is_outlier'] = ((df_clean['QUANTITY'] < lower_bound) | 
                             (df_clean['QUANTITY'] > upper_bound))
    
    outlier_count = df_clean['is_outlier'].sum()
    print(f"Flagged {outlier_count} outliers")
    
    print("\nCleaning complete!")
    return df_clean

# Run the complete analysis
print("Starting data cleaning process...")
df_temp = check_missing_values(df)
duplicates = check_duplicates(df)
outliers = check_outliers(df)
df_clean = clean_data_step_by_step(df)

Starting data cleaning process...

=== Missing Values Analysis ===

Before cleaning:
Missing values in each column:
DATE                   0
ANONYMIZED CATEGORY    0
ANONYMIZED PRODUCT     0
ANONYMIZED BUSINESS    0
ANONYMIZED LOCATION    0
QUANTITY               0
VALUE                  8
dtype: int64

=== Duplicates Analysis ===

Total number of duplicate rows: 3524

First few duplicate rows:
                            DATE ANONYMIZED CATEGORY ANONYMIZED PRODUCT  \
6153   January 6, 2024, 11:52 AM         Category-91       Product-1b48   
7554       July 9, 2024, 2:26 PM        Category-104       Product-af50   
7555       July 9, 2024, 2:26 PM         Category-92       Product-d09a   
12238    April 19, 2024, 3:19 PM         Category-75       Product-086d   
12239    April 19, 2024, 3:19 PM        Category-106       Product-21f4   

      ANONYMIZED BUSINESS ANONYMIZED LOCATION  QUANTITY  VALUE  
6153        Business-20fc       Location-b125         1  3,680  
7554        Business-

In [31]:

def verify_cleaning(df_original, df_cleaned):
    print("\n=== Cleaning Verification Report ===")
    
    # 1. Compare row counts
    print("\n1. Row Count Comparison:")
    print(f"Original rows: {len(df_original)}")
    print(f"Cleaned rows: {len(df_cleaned)}")
    print(f"Rows removed: {len(df_original) - len(df_cleaned)}")
    
    # 2. Check if missing values were handled
    print("\n2. Missing Values Check:")
    print("\nBefore cleaning:")
    print(df_original.isnull().sum())
    print("\nAfter cleaning:")
    print(df_cleaned.isnull().sum())
    
    # 3. Check for duplicates
    print("\n3. Duplicates Check:")
    print(f"Original duplicates: {df_original.duplicated().sum()}")
    print(f"Remaining duplicates: {df_cleaned.duplicated().sum()}")
    
    # 4. Compare QUANTITY statistics
    print("\n4. QUANTITY Statistics Comparison:")
    print("\nBefore cleaning:")
    print(df_original['QUANTITY'].describe())
    print("\nAfter cleaning:")
    print(df_cleaned['QUANTITY'].describe())
    
    # 5. Check outlier flags
    if 'is_outlier' in df_cleaned.columns:
        print("\n5. Outlier Flags:")
        outlier_count = df_cleaned['is_outlier'].sum()
        print(f"Number of rows flagged as outliers: {outlier_count}")
        print(f"Percentage of outliers: {(outlier_count/len(df_cleaned))*100:.2f}%")
    
    # 6. Check VALUE column format
    print("\n6. VALUE Column Format:")
    print("\nBefore cleaning (first 5 unique values):")
    print(df_original['VALUE'].unique()[:5])
    print("\nAfter cleaning (first 5 unique values):")
    print(df_cleaned['VALUE'].unique()[:5])

# Run the verification
verify_cleaning(df, df_clean)


=== Cleaning Verification Report ===

1. Row Count Comparison:
Original rows: 333405
Cleaned rows: 329881
Rows removed: 3524

2. Missing Values Check:

Before cleaning:
DATE                   0
ANONYMIZED CATEGORY    0
ANONYMIZED PRODUCT     0
ANONYMIZED BUSINESS    0
ANONYMIZED LOCATION    0
QUANTITY               0
VALUE                  8
dtype: int64

After cleaning:
DATE                   0
ANONYMIZED CATEGORY    0
ANONYMIZED PRODUCT     0
ANONYMIZED BUSINESS    0
ANONYMIZED LOCATION    0
QUANTITY               0
VALUE                  8
is_outlier             0
dtype: int64

3. Duplicates Check:
Original duplicates: 3524
Remaining duplicates: 0

4. QUANTITY Statistics Comparison:

Before cleaning:
count    333405.000000
mean          2.321186
std           3.790614
min           0.000000
25%           1.000000
50%           1.000000
75%           2.000000
max         359.000000
Name: QUANTITY, dtype: float64

After cleaning:
count    329881.000000
mean          2.321507
std     

In [32]:
def final_cleaning(df):
    print("\n=== Final Cleaning Steps ===")
    df_clean = df.copy()
    
    # 1. Properly convert VALUE to numeric
    print("Converting VALUE to numeric...")
    df_clean['VALUE'] = df_clean['VALUE'].str.replace(',', '').astype(float)
    
    # 2. Handle missing values in VALUE
    print("Handling missing values...")
    value_median = df_clean['VALUE'].median()
    df_clean['VALUE'] = df_clean['VALUE'].fillna(value_median)
    
    # 3. Remove duplicates (already done but included for completeness)
    print("Removing duplicates...")
    df_clean = df_clean.drop_duplicates()
    
    # 4. Flag outliers (already done but included for completeness)
    Q1 = df_clean['QUANTITY'].quantile(0.25)
    Q3 = df_clean['QUANTITY'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_clean['is_outlier'] = ((df_clean['QUANTITY'] < lower_bound) | 
                             (df_clean['QUANTITY'] > upper_bound))
    
    print("\nCleaning complete! Let's verify the results...")
    
    # Quick verification
    print("\nVerification:")
    print(f"Missing values remaining: {df_clean.isnull().sum().sum()}")
    print(f"VALUE dtype: {df_clean['VALUE'].dtype}")
    print(f"Sample of VALUE column (first 5):")
    print(df_clean['VALUE'].head())
    
    return df_clean

# Run the final cleaning
df_final = final_cleaning(df_clean)


=== Final Cleaning Steps ===
Converting VALUE to numeric...
Handling missing values...
Removing duplicates...

Cleaning complete! Let's verify the results...

Verification:
Missing values remaining: 0
VALUE dtype: float64
Sample of VALUE column (first 5):
0     850.0
1    1910.0
2    3670.0
3    2605.0
4    1480.0
Name: VALUE, dtype: float64


In [33]:
def summarize_clean_data(df):
    print("\n=== Clean Data Summary ===")
    
    print("\n1. Dataset Shape:")
    print(f"Rows: {len(df)}")
    print(f"Columns: {len(df.columns)}")
    
    print("\n2. Data Types:")
    print(df.dtypes)
    
    print("\n3. Value Ranges:")
    numeric_cols = ['QUANTITY', 'VALUE']
    for col in numeric_cols:
        print(f"\n{col}:")
        print(df[col].describe())
    
    print("\n4. Outlier Distribution:")
    print("Normal transactions:", len(df[~df['is_outlier']]))
    print("Outlier transactions:", len(df[df['is_outlier']]))
    
    return df

# Run summary
df_summary = summarize_clean_data(df_final)


=== Clean Data Summary ===

1. Dataset Shape:
Rows: 329881
Columns: 8

2. Data Types:
DATE                    object
ANONYMIZED CATEGORY     object
ANONYMIZED PRODUCT      object
ANONYMIZED BUSINESS     object
ANONYMIZED LOCATION     object
QUANTITY                 int64
VALUE                  float64
is_outlier                bool
dtype: object

3. Value Ranges:

QUANTITY:
count    329881.000000
mean          2.321507
std           3.767796
min           0.000000
25%           1.000000
50%           1.000000
75%           2.000000
max         359.000000
Name: QUANTITY, dtype: float64

VALUE:
count    329881.000000
mean       2319.004962
std        1582.561268
min           0.000000
25%        1420.000000
50%        1840.000000
75%        2750.000000
max       16136.000000
Name: VALUE, dtype: float64

4. Outlier Distribution:
Normal transactions: 281766
Outlier transactions: 48115


FEATURE ENGINEERING


In [35]:
import pandas as pd
import numpy as np

def create_time_features(df):
    print("\n=== Creating Time-Based Features ===")
    
    # Convert DATE to datetime if not already
    df['DATE'] = pd.to_datetime(df['DATE'])
    
    # Extract time components
    df['year'] = df['DATE'].dt.year
    df['month'] = df['DATE'].dt.month
    df['day'] = df['DATE'].dt.day
    df['day_of_week'] = df['DATE'].dt.dayofweek  # 0=Monday, 6=Sunday
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    df['quarter'] = df['DATE'].dt.quarter
    
    print("Created time features: year, month, day, day_of_week, is_weekend, quarter")
    return df

def create_transaction_features(df):
    print("\n=== Creating Transaction Features ===")
    
    # Price per unit
    df['price_per_unit'] = df['VALUE'] / df['QUANTITY']
    
    # Transaction size categories
    df['transaction_size'] = pd.qcut(df['VALUE'], q=5, labels=['Very Small', 'Small', 'Medium', 'Large', 'Very Large'])
    
    # Order size categories
    df['order_size'] = pd.qcut(df['QUANTITY'], q=3, labels=['Small', 'Medium', 'Large'])
    
    print("Created features: price_per_unit, transaction_size, order_size")
    return df

def create_business_features(df):
    print("\n=== Creating Business Features ===")
    
    # Business frequency
    business_freq = df.groupby('ANONYMIZED BUSINESS').size()
    df['business_frequency'] = df['ANONYMIZED BUSINESS'].map(business_freq)
    
    # Business average transaction value
    business_avg_value = df.groupby('ANONYMIZED BUSINESS')['VALUE'].mean()
    df['business_avg_value'] = df['ANONYMIZED BUSINESS'].map(business_avg_value)
    
    print("Created features: business_frequency, business_avg_value")
    return df

def create_product_features(df):
    print("\n=== Creating Product Features ===")
    
    # Product popularity
    product_freq = df.groupby('ANONYMIZED PRODUCT').size()
    df['product_popularity'] = df['ANONYMIZED PRODUCT'].map(product_freq)
    
    # Product average price
    product_avg_price = df.groupby('ANONYMIZED PRODUCT')['price_per_unit'].mean()
    df['product_avg_price'] = df['ANONYMIZED PRODUCT'].map(product_avg_price)
    
    print("Created features: product_popularity, product_avg_price")
    return df

def create_location_features(df):
    print("\n=== Creating Location Features ===")
    
    # Location transaction frequency
    location_freq = df.groupby('ANONYMIZED LOCATION').size()
    df['location_frequency'] = df['ANONYMIZED LOCATION'].map(location_freq)
    
    # Location average transaction value
    location_avg_value = df.groupby('ANONYMIZED LOCATION')['VALUE'].mean()
    df['location_avg_value'] = df['ANONYMIZED LOCATION'].map(location_avg_value)
    
    print("Created features: location_frequency, location_avg_value")
    return df

def feature_engineering(df):
    print("Starting feature engineering process...")
    df_featured = df.copy()
    
    # Apply all feature engineering steps
    df_featured = create_time_features(df_featured)
    df_featured = create_transaction_features(df_featured)
    df_featured = create_business_features(df_featured)
    df_featured = create_product_features(df_featured)
    df_featured = create_location_features(df_featured)
    
    # Show new features summary
    print("\n=== New Features Summary ===")
    print("\nNew columns added:")
    new_columns = set(df_featured.columns) - set(df.columns)
    print(sorted(list(new_columns)))
    
    print("\nDataset shape:")
    print(f"Original: {df.shape}")
    print(f"After feature engineering: {df_featured.shape}")
    
    return df_featured

# Run feature engineering
df_with_features = feature_engineering(df_final)

Starting feature engineering process...

=== Creating Time-Based Features ===


C:\Users\BENNLANGAT\AppData\Local\Temp\ipykernel_8992\2066435689.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DATE'] = pd.to_datetime(df['DATE'])


Created time features: year, month, day, day_of_week, is_weekend, quarter

=== Creating Transaction Features ===
Created features: price_per_unit, transaction_size, order_size

=== Creating Business Features ===
Created features: business_frequency, business_avg_value

=== Creating Product Features ===
Created features: product_popularity, product_avg_price

=== Creating Location Features ===
Created features: location_frequency, location_avg_value

=== New Features Summary ===

New columns added:
['business_avg_value', 'business_frequency', 'day', 'day_of_week', 'is_weekend', 'location_avg_value', 'location_frequency', 'month', 'order_size', 'price_per_unit', 'product_avg_price', 'product_popularity', 'quarter', 'transaction_size', 'year']

Dataset shape:
Original: (329881, 8)
After feature engineering: (329881, 23)


In [36]:
def analyze_new_features(df):
    print("\n=== New Features Analysis ===")
    
    # 1. Time-based patterns
    print("\nTransactions by day of week:")
    print(df.groupby('day_of_week')['VALUE'].agg(['count', 'mean']))
    
    print("\nWeekday vs Weekend transactions:")
    print(df.groupby('is_weekend')['VALUE'].agg(['count', 'mean']))
    
    # 2. Transaction size distribution
    print("\nTransaction size distribution:")
    print(df['transaction_size'].value_counts(normalize=True))
    
    # 3. Price per unit statistics
    print("\nPrice per unit statistics:")
    print(df['price_per_unit'].describe())
    
    # 4. Location analysis
    print("\nTop 5 locations by frequency:")
    print(df['location_frequency'].sort_values(ascending=False).head())
    
    # 5. Product popularity
    print("\nTop 5 most popular products:")
    print(df['product_popularity'].sort_values(ascending=False).head())

# Run analysis
analyze_new_features(df_with_features)


=== New Features Analysis ===

Transactions by day of week:
             count         mean
day_of_week                    
0            58037  2275.572859
1            52028  2362.002076
2            50666  2319.359097
3            54499  2348.988018
4            55990  2318.728076
5             2315  2242.573218
6            56346  2298.135129

Weekday vs Weekend transactions:
             count         mean
is_weekend                     
0           271220  2323.993057
1            58661  2295.942432

Transaction size distribution:
transaction_size
Large         0.204177
Small         0.203734
Very Small    0.200027
Medium        0.196556
Very Large    0.195507
Name: proportion, dtype: float64

Price per unit statistics:
count    329881.0
mean          inf
std           NaN
min           0.0
25%         657.5
50%        1380.0
75%        2130.0
max           inf
Name: price_per_unit, dtype: float64

Top 5 locations by frequency:
88979     43891
67792     43891
255860    43891
2558

In [39]:
# Check the first few dates to see their format
print("Sample dates from my dataset:")
print(df_final['DATE'].head())

Sample dates from my dataset:
0   2024-08-18 21:32:00
1   2024-08-18 21:32:00
2   2024-08-18 21:32:00
3   2024-08-18 21:32:00
4   2024-08-18 21:32:00
Name: DATE, dtype: datetime64[ns]


In [41]:
def create_month_year_feature(df):
    print("\n=== Creating Month-Year Feature ===")
    
    # Create Month-Year column (no need to convert DATE as it's already datetime)
    df['Month-Year'] = df['DATE'].dt.strftime('%B %Y')
    
    # Display sample results
    print("\nSample of the new Month-Year feature:")
    display_cols = ['DATE', 'Month-Year']
    print(df[display_cols].head(10))
    
    # Show unique Month-Year values to verify
    print("\nUnique Month-Year values in the dataset:")
    print(sorted(df['Month-Year'].unique()))
    
    return df

# Run the feature engineering
df_with_month_year = create_month_year_feature(df_final)


=== Creating Month-Year Feature ===

Sample of the new Month-Year feature:
                 DATE   Month-Year
0 2024-08-18 21:32:00  August 2024
1 2024-08-18 21:32:00  August 2024
2 2024-08-18 21:32:00  August 2024
3 2024-08-18 21:32:00  August 2024
4 2024-08-18 21:32:00  August 2024
5 2024-08-18 21:32:00  August 2024
6 2024-08-18 21:32:00  August 2024
7 2024-08-18 21:32:00  August 2024
8 2024-08-18 21:32:00  August 2024
9 2024-08-18 21:32:00  August 2024

Unique Month-Year values in the dataset:
['April 2024', 'August 2024', 'December 2024', 'February 2024', 'January 2024', 'July 2024', 'June 2024', 'March 2024', 'May 2024', 'November 2024', 'October 2024', 'September 2024']


In [42]:
print("=== Verification of Data Cleaning and Feature Engineering ===")

# 1. Basic Info
print("\nDataset Shape:", df_with_month_year.shape)

# 2. Check for missing values
missing = df_with_month_year.isnull().sum()
print("\nMissing Values:")
print(missing[missing > 0] if missing.any() > 0 else "No missing values")

# 3. Check for duplicates
duplicates = df_with_month_year.duplicated().sum()
print("\nDuplicate Rows:", duplicates)

# 4. Show engineered Month-Year feature
print("\nSample of Month-Year feature (first 5 rows):")
print(df_with_month_year[['DATE', 'Month-Year']].head())

# 5. Show all columns to confirm feature engineering
print("\nAll Columns in Dataset:")
print(df_with_month_year.columns.tolist())

=== Verification of Data Cleaning and Feature Engineering ===

Dataset Shape: (329881, 9)

Missing Values:
No missing values

Duplicate Rows: 0

Sample of Month-Year feature (first 5 rows):
                 DATE   Month-Year
0 2024-08-18 21:32:00  August 2024
1 2024-08-18 21:32:00  August 2024
2 2024-08-18 21:32:00  August 2024
3 2024-08-18 21:32:00  August 2024
4 2024-08-18 21:32:00  August 2024

All Columns in Dataset:
['DATE', 'ANONYMIZED CATEGORY', 'ANONYMIZED PRODUCT', 'ANONYMIZED BUSINESS', 'ANONYMIZED LOCATION', 'QUANTITY', 'VALUE', 'is_outlier', 'Month-Year']


In [43]:
# Save 
df_with_month_year.to_csv('transaction_data_cleaned_and_engineered.csv', index=False)

print("Dataset has been saved successfully!")
print(f"\nFile name: transaction_data_cleaned_and_engineered.csv")
print(f"Number of rows: {len(df_with_month_year)}")
print(f"Number of columns: {len(df_with_month_year.columns)}")
print("\nColumns saved:")
print(df_with_month_year.columns.tolist())

Dataset has been saved successfully!

File name: transaction_data_cleaned_and_engineered.csv
Number of rows: 329881
Number of columns: 9

Columns saved:
['DATE', 'ANONYMIZED CATEGORY', 'ANONYMIZED PRODUCT', 'ANONYMIZED BUSINESS', 'ANONYMIZED LOCATION', 'QUANTITY', 'VALUE', 'is_outlier', 'Month-Year']
